In [137]:
import signal
import sys
import os
import time
import base64
import random
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from hashlib import sha256
# import secret

In [3]:
sys.dont_write_bytecode = True
BS = 16
pad = lambda s: s + (BS - len(s) % BS) * chr(BS - len(s) % BS)
unpad = lambda s: s[:-ord(s[-1])]
FLAG = 'okFlag'
userAndPw = ['123:456','admin:777']

In [244]:
def alarm(time):
    def handler(signum, frame):
        print('Timeout')
        exit()
    signal.signal(signal.SIGALRM, handler)
    signal.alarm(time)
    
def sha256(content):
    Sha256 = SHA256.new()
    Sha256.update(content)
    return Sha256.digest()

def checkIntegrity(message, digest):
    sha256 = SHA256.new()
    sha256.update(message.encode())
    if base64.b64encode(sha256.digest()) != digest:
        print('Integrity error!')
        exit()
def findPW(ID):
    password = ''
    for each in userAndPw:
        if each.split(':')[0] == ID:
            password = each.split(':')[1]
    if password == '':
        print('ID not found!')
        exit()
    return password

In [5]:
def first():
    try:
        recv = input('You should send your ID and a random string to me: ')
        ID, N, digest = recv.split('||')
    except:
        print('Format error!')
        exit()
    checkIntegrity(ID + '||' + N, digest)
    return ID, N

In [165]:
def second(ID, Nc, password, Ns):
    cipher = sha256(password + '||' + ID + '||' + Nc + '||' + "login")
    print(Ns + '||' + base64.b64encode(cipher))

In [7]:
def third(ID, password, Ns):
    try:
        recv = input('You send me your ID, my random string and your action in base64encode'+
            'and the Mac\n'+
            'Like this Base64encode(ID+"||"+Ns+"||"+action)+"||"+Base64encode(sha256(password+"||"+ID+"||"+Ns+"||"+action))')
        Message,Mac = recv.split('||') #b64decode=>pumptool=>b64encode || base64.b64encode(cipher)<--不能改
        #'admin'||'123'||'login' -s cipherhash --data 'admin'+'||' + '789' + "||"+'login' -a '||printflag' -k 17
        #admin||59419||login 19  ||printflag
    except:
        print('Format error!')
        exit()
    Message = base64.b64decode(Message)
    checkIntegrity(password + "||" + Message, Mac)   # mac= b64encode(sha256(password||Mesage))
    plaintext = Message.split("||")

    if plaintext[0] == ID and plaintext[1] == Ns:
        print('Welcome! ' + ID)
        if ID == 'admin' and plaintext[-1] == "printflag":
            print('The flag is: ' + FLAG)
    else:
        print("Something is wrong")
        exit()

In [31]:
if __name__ == '__main__':
#     alarm(6000000)
#     sys.stdout = os.fdopen(sys.stdout.fileno(), 'w', 0)
#     sys.stdin = os.fdopen(sys.stdin.fileno(), 'r', 0)

    print('Let me sleep first . . .')
#     time.sleep(5)
    print('I\'m back!')

    ID, Nc = first() #N
    password = findPW(ID)
    Ns = str(random.randint(100, 99999))
    second(ID, Nc, password, Ns)
    third(ID, password, Ns)

Let me sleep first . . .
I'm back!
You should send your ID and a random string to me: admin||789||DDn5LzBAmZMlRopSGoaB3+J/14TTnA5iaLywsBdQauQ=
Integrity error!


TypeError: Unicode-objects must be encoded before hashing

***

In [23]:
import base64
import hashlib

In [13]:
recv = 'admin'+'||'+'789'+'||'+'DDn5LzBAmZMlRopSGoaB3+J/14TTnA5iaLywsBdQauQ=' #Nc
ID, N, digest = recv.split('||')
message=ID + '||' + N
message = message.encode()
sha256 = SHA256.new()
sha256.update(message)
print(base64.b64encode(sha256.digest()))

# hashes = hashlib.sha256(message).digest()
# hash_base64 = base64.b64encode(hashes).decode()
# print( 'sha256-{}'.format(hash_base64))

b'DDn5LzBAmZMlRopSGoaB3+J/14TTnA5iaLywsBdQauQ='


In [21]:
mss = 'admin'+'||'+'61325'+'||'+'printflag'
msg = base64.b64encode(mss.encode())
mss2 ='+e3AB/mgf0yzcjTEjUvO6XXZSX1r9L+3h8Z6zQwMF4o='
msg2 = base64.b64encode(mss2.encode())
print(msg) # +'||'+
print(msg2)

b'YWRtaW58fDYxMzI1fHxwcmludGZsYWc='
b'K2UzQUIvbWdmMHl6Y2pURWpVdk82WFhaU1gxcjlMKzNoOFo2elF3TUY0bz0='


In [127]:
ms2 = 'zyAOfrmDlom7iF2ljegu9/dK+nfb5CXISWLI5L5X89Y='
msg3 = base64.b64decode(ms2.encode())
mss = msg3.hex()
print(mss)
# kk= 'f9edc007f9a07f4cb37234c48d4bcee975d9497d6bf4bfb787c67acd0c0c178a'
# len(kk)
# len('7061dca407ae7ecdf74ec1c158afa2703fb86159c973d232eb3b4e80d51d9829') #cf200e7eb9839689bb885da58de82ef7f74afa77dbe425c84962c8e4be57f3d6
# mss2 = bytes.fromhex('1873b29ad99a2ae8ac750287a2264cf693727be7b094b2054af4a7e5377837e9')
mss2 = 'admin||9965\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x10||printflag'
msg4 = base64.b64encode(mss2.encode())
# len(mss2)
msg4 # NzA2MWRjYTQwN2FlN2VjZGY3NGVjMWMxNThhZmEyNzAzZmI4NjE1OWM5NzNkMjMyZWIzYjRlODBkNTFkOTgyOQ==

cf200e7eb9839689bb885da58de82ef7f74afa77dbe425c84962c8e4be57f3d6


52

In [15]:
mm = [0,1,2]
mm[-1]

2

In [47]:
sha256 = SHA256.new()
messages='admin||12345'
sha256.update(messages.encode())
# ci
base64.b64encode(sha256.digest())# == b'GLe7b4Lqp+CGBzoN7TxAq0ltlQZu5phFYqDiAG3OGNE='
# sha256('password' + '||' + 'admin' + '||' + '789' + '||' + "login")

b'zyAOfrmDlom7iF2ljegu9/dK+nfb5CXISWLI5L5X89Y='

In [34]:
from tqdm import tqdm

In [37]:
for i in range(9999999):
    messages = str(i)+'||' + 'admin' + '||' + '789' + '||' + "login"
    sha256 = SHA256.new()
    sha256.update(messages.encode())
    if base64.b64encode(sha256.digest()) == b'+e3AB/mgf0yzcjTEjUvO6XXZSX1r9L+3h8Z6zQwMF4o=':
        print(i)

In [91]:
# from subprocess import *
import command
import socket
from subprocess import *
import subprocess

In [255]:
for length in range(1,31):
    s = socket.create_connection(('140.112.31.96', 10122))
    print(length, '-------------------------------------------')
    time.sleep(7)
    print(s.recv(1000))
    time.sleep(1)
    s.send(b'admin||12345||zyAOfrmDlom7iF2ljegu9/dK+nfb5CXISWLI5L5X89Y=\n')
    time.sleep(1)
    strs = s.recv(1000)
    print(strs)
    Ns = int(str(strs).split('||')[0].split('\'')[-1])
    cipher = b'nhhfpe1pnPDDQFJ7x98n3r+71byd4yrceLgGzDFodQ4='
    args = '-s ' + '9e185fa5ed699cf0c340527bc7df27debfbbd5bc9de32adc78b806cc3168750e' + ' --data ' + '\'||admin||12345||login\''+' -a ' + '\'||printflag\'' + ' -k ' + str(length)
    output = subprocess.getstatusoutput('hashpump ' + args)
    mss2 = output[1].split('\n')[1]
    cipher = output[1].split('\n')[0]
    cipher = base64.b64encode(bytes.fromhex(cipher))
    msg4 = base64.b64encode(mss2[2:].encode())
    s.send(msg4+b'||'+cipher+b'\n')
    print(msg4+b'||'+cipher+b'\n')
    # try:
    strs = s.recv(1000)
    print(strs)

1 -------------------------------------------
b"Let me sleep first . . .\nI'm back!\nYou should send your ID and a random string to me: "
b'37197||nhhfpe1pnPDDQFJ7x98n3r+71byd4yrceLgGzDFodQ4=\nYou send me your ID, my random string and your action in base64encodeand the Mac\nLike this Base64encode(ID+"||"+Ns+"||"+action)+"||"+Base64encode(sha256(password+"||"+ID+"||"+Ns+"||"+action))'
b'YWRtaW58fDEyMzQ1fHxsb2dpblx4ODBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHhiMHx8cHJpbnRmbGFn||hkz/DFPqwHAm5i45CEQYEXaTvxVVl9RWvYXM5yBGw2k=\n'
b'Integrity error!'
2 -------------------------------------------
b"Let me sleep first . . .\nI'm back!\nYou should send your ID and a random string to me: "
b'98140||nhhfpe1pnPDDQFJ7x98n3r+71byd4yrceLgGzDFodQ4=\nYou send me your ID, my random string and your action in base64encodeand the Mac\nLi

b"Let me sleep first . . .\nI'm back!\nYou should send your ID and a random string to me: "
b'85865||nhhfpe1pnPDDQFJ7x98n3r+71byd4yrceLgGzDFodQ4=\nYou send me your ID, my random string and your action in base64encodeand the Mac\nLike this Base64encode(ID+"||"+Ns+"||"+action)+"||"+Base64encode(sha256(password+"||"+ID+"||"+Ns+"||"+action))'
b'YWRtaW58fDEyMzQ1fHxsb2dpblx4ODBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAxXHgxMHx8cHJpbnRmbGFn||hkz/DFPqwHAm5i45CEQYEXaTvxVVl9RWvYXM5yBGw2k=\n'
b'Integrity error!'
14 -------------------------------------------
b"Let me sleep first . . .\nI'm back!\nYou should send your ID and a random string to me: "
b'55766||nhhfpe1pnPDDQFJ7x98n3r+71byd4yrceLgGzDFodQ4=\nYou send me your ID, my random string and your action in base64encodeand the Mac\nLike this Base64encode(ID+"||"+Ns+"||"+action)+"||"+Base64encode(sha256(password+"||"+ID+"||"+Ns+"||"+action))'

b'15574||nhhfpe1pnPDDQFJ7x98n3r+71byd4yrceLgGzDFodQ4=\nYou send me your ID, my random string and your action in base64encodeand the Mac\nLike this Base64encode(ID+"||"+Ns+"||"+action)+"||"+Base64encode(sha256(password+"||"+ID+"||"+Ns+"||"+action))'
b'YWRtaW58fDEyMzQ1fHxsb2dpblx4ODBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDF4fHxwcmludGZsYWc=||hkz/DFPqwHAm5i45CEQYEXaTvxVVl9RWvYXM5yBGw2k=\n'
b'Integrity error!'
27 -------------------------------------------
b"Let me sleep first . . .\nI'm back!\nYou should send your ID and a random string to me: "
b'39600||nhhfpe1pnPDDQFJ7x98n3r+71byd4yrceLgGzDFodQ4=\nYou send me your ID, my random string and your action in base64encodeand the Mac\nLike this Base64encode(ID+"||"+Ns+"||"+action)+"||"+Base64encode(sha256(password+"||"+ID+"||"+Ns+"||"+action))'
b'YWRtaW58fDEyMzQ1fHxsb2dpblx4ODBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMFx4MDBceDAwXHgwMVx4ODB8fHByaW50ZmxhZw==||hkz/DFPqwHAm5i45CEQYEXaTvxVVl9R

In [254]:
# cipher
mss2[2:]
# output

'admin||12345||login\\x80\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x01\\x98||printflag'

In [82]:
s = socket.create_connection(('140.112.31.96', 10122))
time.sleep(7)
print(s.recv(1000))
s.send(b'admin||12345||zyAOfrmDlom7iF2ljegu9/dK+nfb5CXISWLI5L5X89Y=\n')
time.sleep(1)
strs = s.recv(1000)
Ns = int(str(strs).split('||')[0].split('\'')[-1])
cipher = b'nhhfpe1pnPDDQFJ7x98n3r+71byd4yrceLgGzDFodQ4='
args = '-s ' + 'cf200e7eb9839689bb885da58de82ef7f74afa77dbe425c84962c8e4be57f3d6' + ' --data ' + '\'admin||'+str(Ns) +'||\''+' -a ' + '\'||printflag\'' + ' -k ' + str(length)
output = subprocess.getstatusoutput('hashpump ' + args)
mss2 = output[1].split('\n')[1]
msg4 = base64.b64encode(mss2.encode())
s.send(msg4+b'||'+cipher)
# try:
strs = s.recv(1000)

b"Let me sleep first . . .\nI'm back!\nYou should send your ID and a random string to me: "


'b\'59514||nhhfpe1pnPDDQFJ7x98n3r+71byd4yrceLgGzDFodQ4=\\nYou send me your ID, my random string and your action in base64encodeand the Mac\\nLike this Base64encode(ID+"||"+Ns+"||"+action)+"||"+Base64encode(sha256(password+"||"+ID+"||"+Ns+"||"+action))\''

In [ ]:
s.send(b'admin||12345||zyAOfrmDlom7iF2ljegu9/dK+nfb5CXISWLI5L5X89Y=')
s.recv(1000)

In [94]:
# int(str(strs).split('||')[0].split('\'')[-1])
# command.
# return_code = subprocess.call("echo Hello World", shell=True)  
subprocess.getstatusoutput("ls")

(0, 'code6_2.py\ncode6_3.py\ncode6.py\ncode7.ipynb\ncode.ipynb')

In [109]:
Ns=1000
args = '-s ' + 'cf200e7eb9839689bb885da58de82ef7f74afa77dbe425c84962c8e4be57f3d6' + ' --data ' + '\'admin||'+str(Ns) +'||\''+' -a ' + '\'||printflag\'' + ' -k ' + str(29)
output = subprocess.getstatusoutput('hashpump ' + args)
print(output[1])

1873b29ad99a2ae8ac750287a2264cf693727be7b094b2054af4a7e5377837e9
admin||1000||\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01P||printflag


In [220]:
def sha256(msg):
    sha = SHA256.new()
    try:
        sha.update(msg.encode())
    except:
        sha.update(msg)
    return sha.digest()

In [167]:
# output[1].split('\n')[1]
# args
# msg4
# shamac = SHA256.new()
pwd = '56'
ID='admin'
Nc ='12345'
Ns =3389
print(pwd + '||' + ID + '||' + Nc + '||' + "login")
cipher = sha256(pwd + '||' + ID + '||' + Nc + '||' + "login")
# shamac.update(mac.encode())
# print(cipher.encode())
mac = base64.b64encode(cipher)
print(mac)


args = '-s ' + 'dc3cd9c82980289f8cc752cd7275c91c058ccc978b26a5211254a0e80b3ca28b' + ' --data ' + '\'admin||'+str(Ns) +'||\''+' -a ' + '\'||printflag\'' + ' -k ' + str(2)
output = subprocess.getstatusoutput('hashpump ' + args)
mss2 = output[1].split('\n')[1]
msg4 = base64.b64encode(mss2.encode())
#################################################
msg5 = base64.b64decode(msg4)
res = sha256(pwd+'||'+msg5.decode('utf8'))
ress = base64.b64encode(res)
ress
# messages='admin||1000||\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01P||printflag'
# sha256 = SHA256.new()
# sha256.update(messages.encode())
# sha256.digest().hex()

56||admin||12345||login
b'3DzZyCmAKJ+Mx1LNcnXJHAWMzJeLJqUhElSg6As8oos='


b'JVQ+i0j90ldkmZ5BGp1k1ID9gt1fcykPHva6IdKtYQw='

In [162]:
msg5.decode('utf8')

'admin||3389||\\x80\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00x||printflag'

In [252]:
m1 = 'nhhfpe1pnPDDQFJ7x98n3r+71byd4yrceLgGzDFodQ4='
res = base64.b64decode(m1)
print(res.hex())
# ress = sha256(res)
# ress

9e185fa5ed699cf0c340527bc7df27debfbbd5bc9de32adc78b806cc3168750e


In [128]:
mss2 = output[1].split('\n')[1]
msg4 = base64.b64encode(mss2.encode())
len(mss2)

217

In [193]:
sha256('Abc').hex()

'06d90109c8cce34ec0c776950465421e176f08b831a938b3c6e76cb7bee8790b'

In [223]:
sha256('bcdef').hex()

'a2069eb473dc91bedd2048928a74c6f4d390ce07d081e566b469b04efa465623'

In [221]:
pt = 'bc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18def'
sha256(kk).hex()

'db048df0f44d6014004d758b36fe42ba958175994ec5212753e1fc9865ef7033'

In [204]:
args = '-s ' + '06d90109c8cce34ec0c776950465421e176f08b831a938b3c6e76cb7bee8790b' + ' --data ' + '\'bc\''+' -a ' + '\'def\'' + ' -k ' + str(1)
output = subprocess.getstatusoutput('hashpump ' + args)
mss2 = output[1].split('\n')[1]
mss3 = output[1].split('\n')[0]
msg4 = base64.b64encode(mss2.encode())
print(output[1].split('\n')[0])
print(mss2)

50e24cb92bc39bd5a26e10e7a1c3d16e41f5911fd3bc1e3b6b8286d9400c02a4
bc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18def


In [216]:
kk = bytes.fromhex('50e24cb92bc39bd5a26e10e7a1c3d16e41f5911fd3bc1e3b6b8286d9400c02a4')
kk

b'P\xe2L\xb9+\xc3\x9b\xd5\xa2n\x10\xe7\xa1\xc3\xd1nA\xf5\x91\x1f\xd3\xbc\x1e;k\x82\x86\xd9@\x0c\x02\xa4'